In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import proyeccion_rdr.features as features
import proyeccion_rdr.produccion.a03_pabellon as pb

pd.set_option('display.max_columns', None)

# Agrega las columnas de poblacion de interes
ANIOS_INTERES = [str(i) for i in range(2017, 2036)]

2025-05-30 12:03:19.598 | INFO     | proyeccion_rdr.config:<module>:11 - PROJ_ROOT path is: C:\Users\javie\OneDrive\Documents\Trabajo\SSMN\2.0-proyeccion-rdr


## Obtencion de casos por area de influencia RDR

In [3]:
# Lee los casos de todos los macroprocesos
RUTA_ARCHIVOS = "../data/interim/0.1_casos_teoricos_diagnosticos.xlsx"
_, casos_macroproceso = features.leer_casos_macroprocesos(RUTA_ARCHIVOS)

# Obtiene los casos de CMA
casos_cma = casos_macroproceso.query("tipo_paciente == 'cma'")

## Obtencion de porcentajes de CMA

In [4]:
# Define la ruta donde estan los porcentajes de CMA
RUTA_PLANILLA_PORCENTAJES = (
    "../data/raw/3_incidencias_y_porcentajes_marcoprocesos/incidencias_y_prevalencias_RDR.xlsx"
)

# Solamente lee los diagnosticos y el porcentaje de CMA
datos_cma = pd.read_excel(
    RUTA_PLANILLA_PORCENTAJES,
    usecols=["Diagnostico", "Porcentaje Pacientes CMA", "Horas en CMA"],
)

# Deja los diagnosticos que SI aportan a la cantidad de pabellones de CMA (porcentaje > 0)
datos_cma = datos_cma.query("`Porcentaje Pacientes CMA` > 0")

# Formatea la columna de diagnosticos
datos_cma["Diagnostico"] = datos_cma["Diagnostico"].str.split(" - ").str[0]

# Deja como indice a los diagnosticos
datos_cma = datos_cma.set_index("Diagnostico")

# Indica el tiempo quirurgico por diagnostico promedio (o 75%)
tiempos_cma = datos_cma["Horas en CMA"]

In [5]:
# Multiplica los casos en CMA con sus tiempos quirurgicos
horas_en_cma_por_casos = casos_cma[ANIOS_INTERES].mul(tiempos_cma, axis=0)

# Obtiene la cantidad de horas laborales
horas_laborales = features.calcular_horas_laborales(2017, 2035, 12)

# Obtiene cantidad de pabellones de hemodinamia
pabellones_cma = pb.calcular_cantidad_de_pabellones_necesarios(
    horas_en_cma_por_casos, horas_laborales
)

# Obtiene la cantidad de pabellones totales por anio
cantidad_de_pabellones_necesarios = pabellones_cma.sum()

Horas laborales por año calculadas:
+---+------+---------------------------------+
|   | anio | horas_laborales_solo_semana_12h |
+---+------+---------------------------------+
| 0 | 2017 |              2976               |
| 1 | 2018 |              2964               |
| 2 | 2019 |              2988               |
| 3 | 2020 |              3024               |
| 4 | 2021 |              3012               |
+---+------+---------------------------------+

Cantidad de pabellones necesarios calculada:
+-------------+----------------------+---------------------+--------------------+----------------------+--------------------+----------------------+----------------------+---------------------+----------------------+---------------------+---------------------+---------------------+----------------------+---------------------+----------------------+---------------------+----------------------+---------------------+---------------------+
| Diagnostico |         2017         |        2018     

In [6]:
print(f"> Pabellones de CMA 2035: {cantidad_de_pabellones_necesarios['2035']:.2f}")

> Pabellones de CMA 2035: 5.81


## Obtener resumen para MINSAL

In [7]:
# Obtiene porcentaje de hospitalizados
porcentaje_cma = pd.read_excel(
    "../data/raw/3_incidencias_y_porcentajes_marcoprocesos/incidencias_y_prevalencias_RDR.xlsx"
)

porcentaje_cma["Diagnostico"] = porcentaje_cma["Diagnostico"].str.split(" - ").str[0]

# Filtra los porcentajes de hospitalizados
porcentaje_cma = porcentaje_cma.set_index("Diagnostico")
porcentaje_cma = porcentaje_cma["Porcentaje Pacientes CMA"]
porcentaje_cma = porcentaje_cma.dropna()
porcentaje_cma = porcentaje_cma[porcentaje_cma > 0]

In [8]:
# Carga el resumen de area de estudio para MINSAL
resumen_area_de_estudio_minsal = pd.read_excel(
    "../data/interim/0.1_casos_teoricos_diagnosticos.xlsx", sheet_name="resumen_total_RDR"
).sort_values("Diagnostico")
resumen_area_de_estudio_minsal["diagnostico_separado"] = (
    resumen_area_de_estudio_minsal["Diagnostico"].str.split(" - ").str[0]
)
resumen_area_de_estudio_minsal = resumen_area_de_estudio_minsal.set_index("diagnostico_separado")

# Sintetiza informacion de pabellon para el 2035
resumen_quirurgicos = pd.DataFrame(
    {
        "porcentaje_cma": porcentaje_cma,
        "casos_cma_2035": casos_cma["2035"],
        "tiempo_quirurgicos": tiempos_cma,
        "horas_cma_2035": horas_en_cma_por_casos["2035"],
        "horas_laborales_2035_pabellon_12_hrs": horas_laborales["2035"],
        "cantidad_de_pabellones_cma": pabellones_cma["2035"],
    }
)

# Une ambos resumenes segun el diagnostico
resumen_minsal = resumen_area_de_estudio_minsal.merge(
    resumen_quirurgicos, how="inner", left_index=True, right_index=True
)

In [9]:
a_guardar = {
    "resumen_MINSAL": resumen_minsal,
    "casos_cma": casos_cma,
    "resumen_duraciones_cma": tiempos_cma,
    "tiempo_utilizado_pabellon": horas_en_cma_por_casos,
    "horas_laborales_por_anio": horas_laborales,
    "pabellones_desg": pabellones_cma,
    "pabellones_totales": cantidad_de_pabellones_necesarios,
}

In [10]:
with pd.ExcelWriter("../data/interim/2.2_estimacion_pabellones_CMA_RDR.xlsx") as file:
    for nombre_hoja, df_a_guardar in a_guardar.items():
        df_a_guardar.to_excel(file, sheet_name=nombre_hoja)